In [3]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
import math
from sklearn.model_selection import GridSearchCV
import subprocess
from sklearn.utils import shuffle
import itertools
import joblib
from sklearn.feature_selection import *


In [5]:
def make_kmer_list(k, alphabet):
    try:
        return ["".join(e) for e in itertools.product(alphabet, repeat=k)]
    except TypeError:
        print("TypeError: k must be an inter and larger than 0, alphabet must be a string.")
        raise TypeError
    except ValueError:
        print("TypeError: k must be an inter and larger than 0")
        raise ValueError

In [4]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [20]:
import os 
import numpy as np
import pandas as pd
def generating_integrated_training(file_dir,samples_size):   
    Y=[1 for e in xrange(samples_size)]
    Y2=[0 for e in xrange(samples_size)]
    Y.extend(Y2)
    X=[]
    for root, dirs, files in os.walk(file_dir):  
#         print(root) #当前目录路径  
#         print(dirs) #当前路径下所有子目录  
        for e in files:
            print e
            data=pd.read_csv(root+'/'+e,header=None,index_col=False)
            data=data.values[:,2]#设置为probability:2或者predict_label:1
            X.append(data)
    print len(X)
    X=np.array(X).T
    return X,Y
X_,Y=generating_integrated_training('/home02/chenhuangrong/integrated_training',1307)
for select_num in xrange(1,X_.shape[1]):
    F,pval=f_classif(X_,Y)
    idx=np.argsort(F)[::-1]
    select_list=idx[xrange(select_num)]
    X=pd.DataFrame(X_).iloc[:,select_list]
    X=X.values

    pd.DataFrame(X).to_csv('/home02/chenhuangrong/integrated_training_files2.csv',header=None,index=False)
    print select_num
    print select_list
    svc = svm.SVC()
#     parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-5,15,30)), 'gamma':map(lambda x:2**x,np.linspace(-15,5,30))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/%d_gap_%d_gram.csv'%(gap,gram))
    print clf.best_score_
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,),X,Y,cv=10,n_jobs=-1)
    # print type(y_predict)
    # y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
    # predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    # predict_save=np.array(predict_save).T
    # pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/blending_files/frequency_predict_%d_gram.csv'%(gram),header=None,index=False)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    print savedata


RFH_predict.csv
PseDNC_predict.csv
only_3_gram_predict.csv
AthMethPre_independent_test__.csv
PSFNP.csv
PSTNP_2gap.csv
frequency_predict_2_gram.csv
CTD_predict.csv
PSDNP_2gap.csv
only_2_gram_predict.csv
only_5_gram_predict.csv
frequency_predict_4_gram.csv
KNN_predict.csv
PSTNP.csv
only_4_gram_predict.csv
RFH_revised_.csv
GAC_RFH_independent_test__.csv
PCP_predict.csv
frequency_predict_3_gram.csv
PSNP.csv
7_DC.csv
3_DC.csv
PSDNP_1gap.csv
only_6_gram_predict.csv
PSDNP.csv
PSTNP_DS.csv
PSNP_PSDNP_1gap_predict.csv
frequency_predict_6_gram.csv
6_DC.csv
frequency_predict_5_gram.csv
2_DC.csv
PseEIIP_predict.csv
5_DC.csv
4_DC.csv
PSFiveNP.csv
35
1
[28]
0.73221117062
[[['svmC:40.6399743435gamma:0.268584559844', 0.73221117061973984, 0.7311500380807312, 0.7345065034429993, 0.7345065034429993, 0.7299158377964805, 0.7322075729105513, 0.73282442748091614, 0.73282442748091614, 0.46442723498405303, '0.7321', 960.0, 347.0, 353.0, 954.0, 1307.0, 1307.0]]]
2
[28 20]
0.733358837031
[[['svmC:3.72322221568ga

KeyboardInterrupt: 

In [25]:
import os 
import numpy as np
import pandas as pd
def generating_integrated_training(file_dir,samples_size):   
    Y=[1 for e in xrange(samples_size)]
    Y2=[0 for e in xrange(samples_size)]
    Y.extend(Y2)
    X=[]
    for root, dirs, files in os.walk(file_dir):  
#         print(root) #当前目录路径  
#         print(dirs) #当前路径下所有子目录  
        for e in files:
            print e
            data=pd.read_csv(root+'/'+e,header=None,index_col=False)
            data=data.values[:,2]#设置为probability:2或者predict_label:1
            X.append(data)
    print len(X)
    X=np.array(X).T
    return X,Y
X_,Y=generating_integrated_training('/home02/chenhuangrong/integrated_training',1307)
for select_num in xrange(1,X_.shape[1]):
    F,pval=f_classif(X_,Y)
    idx=np.argsort(F)[::-1]
    select_list=idx[xrange(select_num)]
    X=pd.DataFrame(X_).iloc[:,select_list]
    X=X.values

#     pd.DataFrame(X).to_csv('/home02/chenhuangrong/integrated_training_files2.csv',header=None,index=False)
    print select_num
    print select_list
    svc = svm.SVC()
#     parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
#     parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-5,15,30)), 'gamma':map(lambda x:2**x,np.linspace(-15,5,30))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/%d_gap_%d_gram.csv'%(gap,gram))
    print clf.best_score_
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,),X,Y,cv=10,n_jobs=-1)
    # print type(y_predict)
    # y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
    # predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    # predict_save=np.array(predict_save).T
    # pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/blending_files/frequency_predict_%d_gram.csv'%(gram),header=None,index=False)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    print savedata


RFH_predict.csv
PseDNC_predict.csv
only_3_gram_predict.csv
AthMethPre_independent_test__.csv
TD_predict.csv
D_UG_predict.csv
PSFNP.csv
D_CG_predict.csv
PSTNP_2gap.csv
frequency_predict_2_gram.csv
CTD_predict.csv
PSDNP_2gap.csv
only_2_gram_predict.csv
D_AC_predict.csv
only_5_gram_predict.csv
frequency_predict_4_gram.csv
KNN_predict.csv
PSTNP.csv
only_4_gram_predict.csv
RFH_revised_.csv
GAC_RFH_independent_test__.csv
C_predict.csv
PCP_predict.csv
frequency_predict_3_gram.csv
D_predict.csv
PSNP.csv
D_AU_predict.csv
7_DC.csv
D_UC_predict.csv
3_DC.csv
D_AG_predict.csv
CT_predict.csv
PSDNP_1gap.csv
only_6_gram_predict.csv
PSDNP.csv
PSTNP_DS.csv
CD_predict.csv
PSNP_PSDNP_1gap_predict.csv
frequency_predict_6_gram.csv
T_predict.csv
6_DC.csv
frequency_predict_5_gram.csv
2_DC.csv
PseEIIP_predict.csv
5_DC.csv
4_DC.csv
PSFiveNP.csv
47
1
[40]
0.731828615149
[[['svmC:14.2543794902gamma:0.353553390593', 0.73182861514919662, 0.7320061255742726, 0.7314460596786534, 0.7314460596786534, 0.7322111706197398

In [26]:
import os 
import numpy as np
import pandas as pd
def generating_integrated_training(file_dir,samples_size):   
    Y=[1 for e in xrange(samples_size)]
    Y2=[0 for e in xrange(samples_size)]
    Y.extend(Y2)
    X=[]
    for root, dirs, files in os.walk(file_dir):  
#         print(root) #当前目录路径  
#         print(dirs) #当前路径下所有子目录  
        for e in files:
            print e
            data=pd.read_csv(root+'/'+e,header=None,index_col=False)
            data=data.values[:,2]#设置为probability:2或者predict_label:1
            X.append(data)
    print len(X)
    X=np.array(X).T
    return X,Y
X_,Y=generating_integrated_training('/home02/chenhuangrong/integrated_training',1307)
for select_num in xrange(1,X_.shape[1]):
    F,pval=f_classif(X_,Y)
    idx=np.argsort(F)[::-1]
    select_list=idx[xrange(select_num)]
    X=pd.DataFrame(X_).iloc[:,select_list]
    X=X.values

#     pd.DataFrame(X).to_csv('/home02/chenhuangrong/integrated_training_files2.csv',header=None,index=False)
    print select_num
    print select_list
    svc = svm.SVC()
#     parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-5,15,30)), 'gamma':map(lambda x:2**x,np.linspace(-15,5,30))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/%d_gap_%d_gram.csv'%(gap,gram))
    print clf.best_score_
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,),X,Y,cv=10,n_jobs=-1)
    # print type(y_predict)
    # y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
    # predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    # predict_save=np.array(predict_save).T
    # pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/blending_files/frequency_predict_%d_gram.csv'%(gram),header=None,index=False)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    print savedata


RFH_predict.csv
PseDNC_predict.csv
only_3_gram_predict.csv
AthMethPre_independent_test__.csv
TD_predict.csv
D_UG_predict.csv
PSFNP.csv
T_G_U_predict.csv
D_CG_predict.csv
PSTNP_2gap.csv
T_C_G_predict.csv
frequency_predict_2_gram.csv
CTD_predict.csv
PSDNP_2gap.csv
only_2_gram_predict.csv
D_AC_predict.csv
T_C_U_predict.csv
only_5_gram_predict.csv
frequency_predict_4_gram.csv
KNN_predict.csv
PSTNP.csv
only_4_gram_predict.csv
RFH_revised_.csv
GAC_RFH_independent_test__.csv
C_predict.csv
T_A_G_predict.csv
PCP_predict.csv
frequency_predict_3_gram.csv
D_predict.csv
PSNP.csv
D_AU_predict.csv
7_DC.csv
D_UC_predict.csv
3_DC.csv
D_AG_predict.csv
CT_predict.csv
PSDNP_1gap.csv
only_6_gram_predict.csv
PSDNP.csv
PSTNP_DS.csv
CD_predict.csv
PSNP_PSDNP_1gap_predict.csv
frequency_predict_6_gram.csv
T_predict.csv
6_DC.csv
frequency_predict_5_gram.csv
2_DC.csv
PseEIIP_predict.csv
5_DC.csv
T_A_C_predict.csv
4_DC.csv
PSFiveNP.csv
T_A_U_predict.csv
53
1
[44]
0.73221117062
[[['svmC:40.6399743435gamma:0.2685845

KeyboardInterrupt: 

In [4]:
import os 
import numpy as np
import pandas as pd
def generating_integrated_training(file_dir,samples_size):   
    Y=[1 for e in xrange(samples_size)]
    Y2=[0 for e in xrange(samples_size)]
    Y.extend(Y2)
    X=[]
    for root, dirs, files in os.walk(file_dir):  
#         print(root) #当前目录路径  
#         print(dirs) #当前路径下所有子目录  
        for e in files:
            print e
            data=pd.read_csv(root+'/'+e,header=None,index_col=False)
            data=data.values
            positive_=[e for index,e in enumerate(data) if data[index,0] == 1 ]
            negative_=[e for index,e in enumerate(data) if data[index,0] == 0 ]
            positive_.extend(negative_)
            data=np.array(positive_)
            data=data[:,2]#设置为probability:2或者predict_label:1
            X.append(data)
    print len(X)
    X=np.array(X).T
    return X,Y
X_,Y=generating_integrated_training('/home02/chenhuangrong/integrated_training',1307)
for select_num in xrange(1,X_.shape[1]):
    F,pval=f_classif(X_,Y)
    idx=np.argsort(F)[::-1]
    select_list=idx[xrange(select_num)]
    X=pd.DataFrame(X_).iloc[:,select_list]
    X=X.values

#     pd.DataFrame(X).to_csv('/home02/chenhuangrong/integrated_training_files2.csv',header=None,index=False)
    print select_num
    print select_list
    svc = svm.SVC()
#     parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-5,15,30)), 'gamma':map(lambda x:2**x,np.linspace(-15,5,30))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/%d_gap_%d_gram.csv'%(gap,gram))
    print clf.best_score_
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,),X,Y,cv=10,n_jobs=12)
    # print type(y_predict)
    # y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
    # predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    # predict_save=np.array(predict_save).T
    # pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/blending_files/frequency_predict_%d_gram.csv'%(gram),header=None,index=False)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    print savedata


RFH_predict.csv
PseDNC_predict.csv
only_3_gram_predict.csv
AthMethPre_independent_test__.csv
TD_predict.csv
D_UG_predict.csv
PSFNP.csv
T_G_U_predict.csv
D_CG_predict.csv
PSTNP_2gap.csv
T_C_G_predict.csv
frequency_predict_2_gram.csv
CTD_predict.csv
PSDNP_2gap.csv
only_2_gram_predict.csv
D_AC_predict.csv
T_C_U_predict.csv
only_5_gram_predict.csv
frequency_predict_4_gram.csv
KNN_predict.csv
PSTNP.csv
only_4_gram_predict.csv
RFH_revised_.csv
GAC_RFH_independent_test__.csv
C_predict.csv
T_A_G_predict.csv
PCP_predict.csv
frequency_predict_3_gram.csv
D_predict.csv
PSNP.csv
D_AU_predict.csv
7_DC.csv
D_UC_predict.csv
3_DC.csv
D_AG_predict.csv
CT_predict.csv
PSDNP_1gap.csv
only_6_gram_predict.csv
PSDNP.csv
PSTNP_DS.csv
CD_predict.csv
PSNP_PSDNP_1gap_predict.csv
frequency_predict_6_gram.csv
T_predict.csv
6_DC.csv
frequency_predict_5_gram.csv
2_DC.csv
PseEIIP_predict.csv
5_DC.csv
T_A_C_predict.csv
4_DC.csv
PSFiveNP.csv
T_A_U_predict.csv
53
1
[44]
0.73221117062
[[['svmC:40.6399743435gamma:0.2685845

In [8]:
import os 
import numpy as np
import pandas as pd
from sklearn.feature_selection import f_classif
def generating_integrated_training(file_dir,samples_size):   
    Y=[1 for e in xrange(samples_size)]
    Y2=[0 for e in xrange(samples_size)]
    Y.extend(Y2)
    X=[]
    for root, dirs, files in os.walk(file_dir):  
#         print(root) #当前目录路径  
#         print(dirs) #当前路径下所有子目录  
        for e in files:
            if e=="GAC_RFH_independent_test__.csv" or e=="PseEIIP_predict.csv" or e=="PCP_predict.csv" or e=="KNN_predict.csv":
                print e
                data=pd.read_csv(root+'/'+e,header=None,index_col=False)
                data=data.values
                positive_=[e for index,e in enumerate(data) if data[index,0] == 1 ]
                negative_=[e for index,e in enumerate(data) if data[index,0] == 0 ]
                positive_.extend(negative_)
                data=np.array(positive_)
                data=data[:,2]#设置为probability:2或者predict_label:1
                X.append(data)
    print len(X)
    X=np.array(X).T
    return X,Y
X_,Y=generating_integrated_training('/home02/chenhuangrong/integrated_training',1307)
for select_num in xrange(4,X_.shape[1]+1):
    F,pval=f_classif(X_,Y)
    idx=np.argsort(F)[::-1]
    select_list=idx[xrange(select_num)]
    X=pd.DataFrame(X_).iloc[:,select_list]
    X=X.values

#     pd.DataFrame(X).to_csv('/home02/chenhuangrong/integrated_training_files2.csv',header=None,index=False)
    print select_num
    print select_list
    svc = svm.SVC()
#     parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-5,15,30)), 'gamma':map(lambda x:2**x,np.linspace(-15,5,30))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/%d_gap_%d_gram.csv'%(gap,gram))
    print clf.best_score_
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,),X,Y,cv=10,n_jobs=12)
    # print type(y_predict)
    # y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=-1,method='predict_proba')
    # predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    # predict_save=np.array(predict_save).T
    # pd.DataFrame(predict_save).to_csv('/home02/chenhuangrong/blending_files/frequency_predict_%d_gram.csv'%(gram),header=None,index=False)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    print savedata


KNN_predict.csv
GAC_RFH_independent_test__.csv
PCP_predict.csv
PseEIIP_predict.csv
4
4
[1 0 2 3]
0.83779648049
[[['svmC:40.6399743435gamma:0.000537268559055', 0.83779648048967104, 0.8473642800944139, 0.8240244835501148, 0.8240244835501148, 0.8515684774292273, 0.8376832783464027, 0.83553141970519784, 0.83553141970519784, 0.67584938348985335, '0.8377', 1077.0, 230.0, 194.0, 1113.0, 1307.0, 1307.0]]]
